In [1]:
import sys
import os

# Thêm đường dẫn vào sys.path
sys.path.append('/kaggle/input/raindrop_removal/pytorch/default/1/Raindrop-Removal')

https://github.com/Hyukju/Raindrop-Removal 

https://www.mdpi.com/2227-7390/11/15/3318

In [2]:
%cd /kaggle/input/raindrop_removal/pytorch/default/1/Raindrop-Removal

/kaggle/input/raindrop_removal/pytorch/default/1/Raindrop-Removal


In [3]:
import sys
sys.argv = sys.argv[:1]  # Loại bỏ đối số '-f' của Jupyter

## import libaries

In [4]:
import os
import cv2 
import numpy as np 
import torch
from torch.utils.data import Dataset
from torchvision.transforms import transforms

## intialize dataset

In [5]:
class ToTensor():
    def __call__(self, data):
        for k, v in data.items():
            if v.ndim == 2:
                v = v[:,:,np.newaxis]
            data[k] = torch.from_numpy(v.transpose((2,0,1)).astype('float32'))

        return data

class Normalization():
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std 

    def __call__(self, data):
        
        for k, v in data.items():
            data[k] = (v - self.mean) / self.std

        return data


class RandomFlip():
    def __call__(self, data):

        flag_lr = False 
        flag_ud = False 

        if np.random.rand() > 0.5: flag_lr = True
        if np.random.rand() > 0.5: flag_ud = True

        for k, v in data.items():
            if flag_lr: v = np.fliplr(v)
            if flag_ud: v = np.flipud(v)
            data[k] = v

        return data

class RandomCrop():
    def __init__(self, shape):
        self.shape = shape
    
    def __call__(self, data):
        input  = data['input']
        h, w = input.shape[:2]
        new_h, new_w = self.shape

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)
        
        id_y = np.arange(top, top + new_h, 1)[:, np.newaxis]
        id_x = np.arange(left, left + new_w, 1)

        for k, v in data.items():
            data[k] = v[id_y, id_x]

        return data

In [6]:
# class ModelDataset(Dataset):
#     def __init__(self, data_dir, dataA_dir = 'Drop', dataB_dir='Clear', use_transform=False):
#         self.data_dir = data_dir
#         self.use_transform = use_transform
#         self.transform =  transforms.Compose([
#                                             RandomCrop(shape=(256, 256)),
#                                             RandomFlip(),
#                                             Normalization(mean=0.5, std=0.5),
#                                             ToTensor(),
#                                             ])
#         self.dataA_dir = dataA_dir
#         self.dataB_dir = dataB_dir

#         lst_dataA = []
#         for folder in os.listdir(os.path.join(self.data_dir, self.dataA_dir)):
#             folder_path = os.path.join(self.data_dir, self.dataA_dir, folder)
#             if os.path.isdir(folder_path):  # Check if it's a folder
#                 images = sorted([img for img in os.listdir(folder_path) if img.lower().endswith(('jpg', 'png'))])
#                 if images:  # Ensure there is at least one image
#                     lst_dataA.append(os.path.join(folder_path, images[1]))  # Take the first image
#         lst_dataA.sort()

        
#         lst_dataB = []
#         for folder in os.listdir(os.path.join(self.data_dir, self.dataB_dir)):
#             folder_path = os.path.join(self.data_dir, self.dataB_dir, folder)
#             if os.path.isdir(folder_path):  # Check if it's a folder
#                 images = ([img for img in os.listdir(folder_path) if img.lower().endswith(('jpg', 'png'))])
#                 if images:  # Ensure there is at least one image
#                     lst_dataB.append(os.path.join(folder_path, images[1]))  # Take the first image
#         lst_dataB.sort()

#         self.lst_dataA = lst_dataA
#         self.lst_dataB = lst_dataB

#     def __len__(self):
#         return len(self.lst_dataA)

#     def __getitem__(self, index):

#         imgA = cv2.imread(os.path.join(self.data_dir, self.dataA_dir, self.lst_dataA[index]))
#         imgB = cv2.imread(os.path.join(self.data_dir, self.dataB_dir, self.lst_dataB[index]))
#         imgA = cv2.cvtColor(imgA, cv2.COLOR_BGR2RGB)
#         imgB = cv2.cvtColor(imgB, cv2.COLOR_BGR2RGB)

#         if imgA.dtype == np.uint8:
#             imgA = imgA / 255.0  
#         if imgB.dtype == np.uint8:
#             imgB = imgB / 255.0      

#         grayA = cv2.cvtColor(imgA.astype('float32'), cv2.COLOR_RGB2GRAY)
#         grayB = cv2.cvtColor(imgB.astype('float32'), cv2.COLOR_RGB2GRAY)

#         mask = (grayA - grayB) * 0.5 + 0.5
     
#         data = {'input':imgA, 'label':imgB, 'mask':mask}

#         if self.use_transform:
#             data = self.transform(data)

#         return data

In [7]:
import os
import cv2
import torch
import random
import numpy as np
from torch.utils.data import Dataset

class ModelDataset(Dataset):
    def __init__(self, root_dir='/kaggle/input/ntire2025', subset_day="DayRainDrop_Train_img/DayRainDrop_Train", 
                 subset_night="NightRainDrop_Train_img/NightRainDrop_Train", use_transform=False):
        """
        Initialize the dataset with the same path structure as RaindropDataset.
        :param root_dir: Root directory of the dataset
        :param subset_day: Path to the daytime dataset
        :param subset_night: Path to the nighttime dataset
        :param use_transform: Whether to apply transformations
        """
        self.root_dir = root_dir
        self.use_transform = use_transform

        self.transform = transforms.Compose([
            RandomCrop(shape=(256, 256)),
            RandomFlip(),
            Normalization(mean=0.5, std=0.5),
            ToTensor(),
        ])

        # Daytime paths
        subset_path_day = os.path.join(root_dir, subset_day)
        self.drop_path_day = os.path.join(subset_path_day, "Drop")
        self.clear_path_day = os.path.join(subset_path_day, "Clear")

        # Nighttime paths
        subset_path_night = os.path.join(root_dir, subset_night)
        self.drop_path_night = os.path.join(subset_path_night, "Drop")
        self.clear_path_night = os.path.join(subset_path_night, "Clear")

        # Load image lists
        self.lst_dataA_all = []  # Drop images
        self.lst_dataB_all = []  # Clear images

        # Add day images if they exist
        if os.path.exists(self.drop_path_day) and os.path.exists(self.clear_path_day):
            drop_images_day = sorted([os.path.join(self.drop_path_day, img) 
                                   for img in os.listdir(self.drop_path_day)
                                   if img.lower().endswith(('.png', '.jpg', '.jpeg'))])
            clear_images_day = sorted([os.path.join(self.clear_path_day, img) 
                                    for img in os.listdir(self.clear_path_day)
                                    if img.lower().endswith(('.png', '.jpg', '.jpeg'))])
            self.lst_dataA_all.extend(drop_images_day)
            self.lst_dataB_all.extend(clear_images_day)

        # Add night images if they exist
        if os.path.exists(self.drop_path_night) and os.path.exists(self.clear_path_night):
            drop_images_night = sorted([os.path.join(self.drop_path_night, img) 
                                     for img in os.listdir(self.drop_path_night)
                                     if img.lower().endswith(('.png', '.jpg', '.jpeg'))])
            clear_images_night = sorted([os.path.join(self.clear_path_night, img) 
                                      for img in os.listdir(self.clear_path_night)
                                      if img.lower().endswith(('.png', '.jpg', '.jpeg'))])
            self.lst_dataA_all.extend(drop_images_night)
            self.lst_dataB_all.extend(clear_images_night)

        # Ensure the lists have the same length and corresponding pairs
        assert len(self.lst_dataA_all) == len(self.lst_dataB_all), "Mismatch in number of input and label images"

    def __len__(self):
        return len(self.lst_dataA_all)

    def __getitem__(self, index):
        imgA_path = self.lst_dataA_all[index]
        imgB_path = self.lst_dataB_all[index]

        imgA = cv2.imread(imgA_path)
        imgB = cv2.imread(imgB_path)

        if imgA is None or imgB is None:
            raise ValueError(f"Error loading images: {imgA_path} or {imgB_path}")

        imgA = cv2.cvtColor(imgA, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
        imgB = cv2.cvtColor(imgB, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

        grayA = cv2.cvtColor(imgA, cv2.COLOR_RGB2GRAY)
        grayB = cv2.cvtColor(imgB, cv2.COLOR_RGB2GRAY)
        mask = (grayA - grayB) * 0.5 + 0.5

        data = {'input': imgA, 'label': imgB, 'mask': mask}

        if self.use_transform:
            data = self.transform(data)

        return data


In [8]:
import importlib

def find_model(model_name, phase='train', learning_rate=0.002):
    """
    Searches for the specified model inside the 'model' directory.
    Imports the corresponding <model_name>_model.py and instantiates the 'Model' class.
    
    Parameters:
        model_name (str): Name of the model.
        phase (str): Either 'train' or 'test'.
        learning_rate (float): Learning rate for the model.

    Returns:
        model: Instantiated model class or None if not found.
    """
    
    model = None
    model_module_name = f'model.{model_name.lower()}_model'

    print(f'Searching for model: {model_module_name}')
    
    try:
        # Import the model module dynamically
        model_module = importlib.import_module(model_module_name)
    except ModuleNotFoundError:
        print(f'Error: {model_module_name} module not found.')
        return None
    
    target_model_name = 'Model'

    # Search for the class inside the module
    for name, cls in model_module.__dict__.items():
        if name.lower() == target_model_name.lower() and isinstance(cls, type):
            model = cls  # Store the class

    if model is None:
        print(f'Error: {target_model_name} class not found in {model_module_name}.')
        return None

    print(f'{model_module_name}.{target_model_name} successfully created.')
    
    # Instantiate the model with provided parameters
    return model(phase=phase, in_channels=3, out_channels=3, nker=64, lr=learning_rate)

## Train

In [9]:
import os
import argparse
import utils
import torch
import gc
from torch.utils.data import DataLoader
from datetime import datetime
from time import time

def train(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #ckpt_dir = os.path.join(args.ckpt_dir, args.model)
    ckpt_dir=args.ckpt_dir
    os.makedirs(ckpt_dir, exist_ok=True)
    save_dir= args.save_dir
    utils.print_args(args)
    utils.save_log(save_dir, 'train', args)
    
    model = find_model(model_name=args.model, phase='train', learning_rate=args.learning_rate)

    dataset = ModelDataset(
        use_transform=True
    )
    print(len(dataset))
    data_loader = DataLoader(
        dataset=dataset,
        batch_size=args.batch_size,
        shuffle=True,
        num_workers=args.num_workers,
        pin_memory=True  # ✅ Optimizes GPU memory transfer
    )

    max_batches = len(data_loader)
    start_epoch = 1

    # Check for continued training
    if args.train_continue:
        last_epoch = model.load(ckpt_dir,epoch=args.ckpt_epoch)
        if last_epoch > 1:
            start_epoch = last_epoch + 1
            print(f'Continued Training from EPOCH {last_epoch}!!')

    for epoch in range(start_epoch, args.ckpt_epoch+args.max_epochs + 1):
        start_t = time()
        losses_list = []

        for batch, data in enumerate(data_loader, 1):
            # Move data to GPU
            for key in data.keys():
                data[key] = data[key].to(device, non_blocking=True)

            # Forward pass
            model.set_inputs(data)
            model.train_on_batch()  # 🚨 Ensure this function returns loss tensor

            # Backward pass
            #loss.backward()
            #model.optimizer.step()
            #model.optimizer.zero_grad(set_to_none=True)  # ✅ Free memory

            # Print loss for each batch
            if batch == 1 or batch % 10 == 0 or batch == max_batches:
                losses_list.append(model.get_losses())
                utils.print_losses(epoch, args.max_epochs, batch, max_batches, losses_list, title=f'[{args.model}] TRAIN', mode='last')

            # ✅ Free memory after every batch
            del data  # Delete unnecessary variables
            torch.cuda.empty_cache()  # Free GPU cache
            gc.collect()  # Run garbage collection

        # Print loss for each epoch (average of batch losses)
        delay_t = time() - start_t
        utils.print_losses(epoch, args.max_epochs, batch, max_batches, losses_list, title=f'[{args.model}: {delay_t:.3f}s] TRAIN MEAN LOSS', mode='mean')
        utils.save_losses(save_dir, epoch, losses_list, mode='mean')

        # Save images
        utils.save_outputs(
           save_dir=os.path.join(args.save_dir, args.model, 'train'),
           filename=f'{epoch:04d}.png',
           outputs=model.get_outputs(),
           max_display=3
        )

        # Save model every 200 epochs
        if epoch % 2 == 0:
            model.save(save_dir, epoch)

    print(f'Training Finished!!: {datetime.now()}')

if __name__ == '__main__':
    parser = argparse.ArgumentParser(prog='DeRainDrop')             
    
    parser.add_argument('--train_continue', action='store_true', dest='train_continue', default=True)
    parser.add_argument('--model', default='proposed', type=str, dest='model')
    parser.add_argument('--ckpt_epoch', default=580, type=int, dest='ckpt_epoch')
    parser.add_argument('--max_epochs', default=20, type=int, dest='max_epochs')
    parser.add_argument('--batch_size', default=8, type=int, dest='batch_size')
    parser.add_argument('--num_workers', default=8, type=int, dest='num_workers')
    parser.add_argument('--lr', default=0.0001, type=float, dest='learning_rate')
    parser.add_argument('--ckpt_dir', default='/kaggle/input/raindrop-pipeline-train', type=str, dest='ckpt_dir')    
    parser.add_argument('--dataset_dir_day', default='/kaggle/input/ntire2025/DayRainDrop_Train', type=str, dest='dataset_dir_day')
    parser.add_argument('--dataset_dir_night', default='/kaggle/input/ntire2025/NightRainDrop_Train', type=str, dest='dataset_dir_night')    

    parser.add_argument('--save_dir', default='/kaggle/working/', type=str, dest='save_dir')
    
    args = parser.parse_args()
    train(args)

-------------------------------------------
train_continue: True
model: proposed
ckpt_epoch: 580
max_epochs: 20
batch_size: 8
num_workers: 8
learning_rate: 0.0001
ckpt_dir: /kaggle/input/raindrop-pipeline-train
dataset_dir_day: /kaggle/input/ntire2025/DayRainDrop_Train
dataset_dir_night: /kaggle/input/ntire2025/NightRainDrop_Train
save_dir: /kaggle/working/
-------------------------------------------
Searching for model: model.proposed_model
model.proposed_model.Model successfully created.
initialize network with normal
initialize network with normal
initialize network with normal


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 201MB/s]


13368


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/kaggle/input/raindrop_removal/pytorch/default/1/Raindrop-Removal/utils.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that coul

Continued Training from EPOCH 580!!
[proposed] TRAIN: EPOCH 581/20 | BATCH 1/1671 | LOSS D: 0.6921 | LOSS G: 1.1789 | LOSS GAN: 0.7527 | LOSS MULTSCALE: 0.0115 | LOSS SSIM: 0.2639 | LOSS VGG: 0.1507 | LOSS MASK: 0.0029 | 
[proposed] TRAIN: EPOCH 581/20 | BATCH 10/1671 | LOSS D: 0.7042 | LOSS G: 1.0909 | LOSS GAN: 0.6893 | LOSS MULTSCALE: 0.0216 | LOSS SSIM: 0.2684 | LOSS VGG: 0.1116 | LOSS MASK: 0.0045 | 
[proposed] TRAIN: EPOCH 581/20 | BATCH 20/1671 | LOSS D: 0.6999 | LOSS G: 1.1726 | LOSS GAN: 0.7201 | LOSS MULTSCALE: 0.0157 | LOSS SSIM: 0.2675 | LOSS VGG: 0.1692 | LOSS MASK: 0.0046 | 
[proposed] TRAIN: EPOCH 581/20 | BATCH 30/1671 | LOSS D: 0.7171 | LOSS G: 1.0245 | LOSS GAN: 0.6875 | LOSS MULTSCALE: 0.0154 | LOSS SSIM: 0.2287 | LOSS VGG: 0.0930 | LOSS MASK: 0.0044 | 
[proposed] TRAIN: EPOCH 581/20 | BATCH 40/1671 | LOSS D: 0.6915 | LOSS G: 1.1307 | LOSS GAN: 0.7429 | LOSS MULTSCALE: 0.0132 | LOSS SSIM: 0.2516 | LOSS VGG: 0.1230 | LOSS MASK: 0.0033 | 
[proposed] TRAIN: EPOCH 581/20